# Guardian - Iteration 12

## Import Packages

In [1]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [2]:
iteration = 12

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2017-07-01",
    "to-date": "2017-11-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY4"),
}


In [3]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [4]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [5]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2017-07-01&to-date=2017-11-01&order-by=newest&api-key=295c6540-8c34-4390-ad47-0cf94ebc153f&page=1
status ok
total 28417
startIndex 1
pageSize 50
pages 569
orderBy newest
---- RUNTIME STATUS ---- 
(1/569) in 0:00:08.199137s
(2/569) in 0:00:10.422253s
(3/569) in 0:00:13.073975s
(4/569) in 0:00:05.816839s
(5/569) in 0:00:11.237577s
(6/569) in 0:00:10.324812s
(7/569) in 0:00:26.710686s
(8/569) in 0:00:14.985485s
(9/569) in 0:00:07.447092s
(10/569) in 0:00:09.434630s
(11/569) in 0:00:18.100213s
(12/569) in 0:00:06.693813s
(13/569) in 0:00:07.103961s
(14/569) in 0:00:09.414888s
(15/569) in 0:00:21.040842s
(16/569) in 0:00:08.172019s
(17/569) in 0:00:08.410414s
(18/569) in 0:00:20.556146s
(19/569) in 0:01:36.067367s
(20/569) in 0:00:03.252255s
(21/569) in 0:00:12.550024s
(22/569) in 0:00:14.241719s
(23/569) in 0:00:06.794280s
(24/569) in 0:00:08.817217s
(25/569) i

In [6]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}_1.csv.zip", compression="zip")
    return df

In [7]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(28367, 13) took 0:00:00.009074s

unfold_columns:
 shape=(28367, 49) took 0:00:06.306557s

drop_columns:
 shape=(28367, 31) took 0:00:00.017712s

create_csv:
 shape=(28367, 31) took 0:00:21.940084s

